## 임상시험 데이터: DB Specification Sheet 보기
### 목적
DB Specsheet는 CRF 화면 구성뿐만 아니라 모든 데이터셋(sas7bdat/xport)의 컬럼 사양을 포함한다. DB Specsheet를 바탕으로 각 데이터셋의 컬럼 사양을 추출하려 한다.

### 준비
임상시험 데이터: [cafe.naver.com/dmisimportant/104](cafe.naver.com/dmisimportant/104)

### Libraries

In [60]:
import os
import pandas as pd

### Variables
Specsheet, 데이터셋 경로 설정

In [61]:
ornament="-"*10
ext=(".sas7bdat",".xport")
specsheet="C:/code/CUBEDEMO2017/spec.xlsx"
path_set="C:/code/CUBEDEMO2017/SASSET/"

#### 데이터셋: 로드
데이터셋 로드 및 이상 여부 확인

In [62]:
sasobj=[obj for obj in os.scandir(path_set) if any(map(obj.path.lower().__contains__,ext)) and obj.is_file()]
sasbad=[obj for obj in sasobj if obj.stat().st_size<3]
if len(sasbad)>1:raise Exception("exotic file exists")
print(ornament,"number of loaded sas7bdat:",len(sasobj))

def _decode(filepath):
    data=pd.read_sas(filepath)
    nas=data.notna().value_counts().sum()
    bytecol=data.select_dtypes("object").columns
    data[bytecol]=data[bytecol].apply(lambda q:q.str.decode("utf-8"))
    if nas==data.notna().value_counts().sum():
        return data
    else:
        print(ornament,"error:",filepath)
        return None

data={os.path.splitext(obj.name)[0].upper():_decode(obj.path) for obj in sasobj}
print(ornament,"domain:",os.linesep,data.keys(),os.linesep,len(data),"domains")

---------- number of loaded sas7bdat: 27
---------- domain: 
 dict_keys(['AE', 'AY', 'CM', 'CT', 'CY', 'DA', 'DM', 'DS', 'DY', 'EF', 'EG', 'EN', 'ES', 'IE', 'IP', 'LB', 'LC', 'LY', 'MH', 'MY', 'PD', 'PG', 'RN', 'SN', 'SU', 'SV', 'VS']) 
 27 domains


#### 스펙시트: 로드
스펙시트 로드 및 형태 확인
- 100% 0 non-null 컬럼 제거

In [63]:
spec=pd.read_excel(specsheet)
print(spec.info())
spec=spec.dropna(how="all",axis=1)
spec.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DOMAIN           291 non-null    object 
 1   PGNM             291 non-null    object 
 2   PGNO             291 non-null    int64  
 3   PAGE_LABEL       291 non-null    object 
 4   CRF_LABEL        291 non-null    object 
 5   VISIT            291 non-null    object 
 6   ITEMID           291 non-null    object 
 7   ITEM_SEQ         291 non-null    int64  
 8   ITEM_LABEL       291 non-null    object 
 9   CODE             107 non-null    object 
 10  LAYOUT           291 non-null    object 
 11  KEY              113 non-null    float64
 12  TYPE_LENGTH      291 non-null    object 
 13  VIEW_TYPE        291 non-null    object 
 14  DERIVED          0 non-null      float64
 15  DERIVED_EXPLAIN  0 non-null      float64
 16  COMMENT          0 non-null      float64
 17  ISSUE           

Index(['DOMAIN', 'PGNM', 'PGNO', 'PAGE_LABEL', 'CRF_LABEL', 'VISIT', 'ITEMID',
       'ITEM_SEQ', 'ITEM_LABEL', 'CODE', 'LAYOUT', 'KEY', 'TYPE_LENGTH',
       'VIEW_TYPE'],
      dtype='object')

#### 스펙시트: 컬럼 정의 대상 파악
스펙시트는 CRScube 사양이라고 가정
- 스펙시트의 모든 컬럼이 CDMS/CRF/Validation에 사용됨
- 스펙시트 컬럼명과 데이터셋 컬럼명이 다를뿐 데이터셋의 모든 컬럼은 스펙시트 내 정의됨

In [64]:
dataset_col=[]
[dataset_col.extend(q) for q in [w.columns for w in data.values()]]
dataset_col_wo_spec=[q for q in dataset_col if not q in spec.ITEMID.unique()]
print(ornament,"dataset column without spec.:",len(dataset_col_wo_spec),dataset_col_wo_spec)

---------- dataset column without spec.: 0 []


#### 스펙시트와 데이터셋의 우선 대응 컬럼 파악
데이터셋은 [DOMAIN..VARNAME]로서 스펙시트에 대응
- 먼저 DOMAIN으로 시작/쿼리하며 스펙시트의 viewport 및 attribute를 내고자 함

In [65]:
mh=data["MH"]
spec[spec.DOMAIN=="MH"].head(5)

,DOMAIN,PGNM,PGNO,PAGE_LABEL,CRF_LABEL,VISIT,ITEMID,ITEM_SEQ,ITEM_LABEL,CODE,LAYOUT,KEY,TYPE_LENGTH,VIEW_TYPE
21,MH,MH,6,Medical History,Medical History,1,SUBJID,1,Screening Number,NaN,SYSDEFINED,1.0,C8,nvarchar2(8)
22,MH,MH,6,Medical History,Medical History,1,SEQ,2,Seq,NaN,SYSDEFINED,1.0,N,num
23,MH,MH,6,Medical History,Medical History,1,MHTERM,3,Medical history term,NaN,MEDCOD,NaN,C255,nvarchar2(255)
24,MH,MH,6,Medical History,Medical History,1,MHONGO,4,Ongoing,1:Yes|2:No,RADIO,NaN,N2,num
25,MH,MH,6,Medical History,Medical History,1,MHENDTC,5,End date,NaN,DATE,NaN,YYYY-UK-UK,nvarchar2(10)


- DOMAIN과 PAGE_LABEL, CRF_LABEL은 pair이나 이외는 VARNAME에 대응함
- 이에, per-DOMAIN spec을 만들고 거기서 per-VARNAME spec을 내고자 함

In [66]:
class specs:
    def __init__(self,spec):
        try:
            spec["CODE"]=spec.CODE.apply(lambda q:dict(item.split(":") for item in q.split("|")) if pd.notna(q) else q).dropna()
        except Exception as err:
            print(err)
        self.data={domain:spec[spec.DOMAIN==domain] for domain in spec.DOMAIN.unique()}
    def bydom(self,domain):
        self.spec=self.data[domain]
        return self.spec
    def byvar(self,domain,varname):
        spec=self.bydom(domain)
        spec=spec[spec.ITEMID==varname].iloc[0]
        _spec=dict(zip(spec.index,spec))
        # setattr(q,w,e)
        self.domain=domain
        self.crf=_spec["CRF_LABEL"]
        self.visit=_spec["VISIT"]
        self.var=_spec["ITEMID"]
        self.varname=_spec["ITEM_LABEL"]
        self.code=_spec["CODE"]
        self.type=_spec["TYPE_LENGTH"]
        return spec
    
spec=specs(spec)
spec.bydom("MH")
spec.byvar("MH","SUBJID")

DOMAIN                       MH
PGNM                         MH
PGNO                          6
PAGE_LABEL      Medical History
CRF_LABEL       Medical History
VISIT                         1
ITEMID                   SUBJID
ITEM_SEQ                      1
ITEM_LABEL     Screening Number
CODE                        NaN
LAYOUT               SYSDEFINED
KEY                         1.0
TYPE_LENGTH                  C8
VIEW_TYPE          nvarchar2(8)
Name: 21, dtype: object

- DOMAIN, VARNAME(ITEMID)의 정의를 딕셔너리 또는 attribute로 냄
- CODE에 따라 코딩 또는 디코딩 가능
- TYPE_LENGTH(CRF 상 edit check 용), VIEW_TYPE(DB 상 type)로 기본적 무결성 확인이 가능
    - 예컨대 SUBJID는 C8(character 8)이어야 함

In [72]:
def _get_ect(type_length):
    type=type_length[0]
    if type=="C":
        return str,int(type_length[1:])
    elif type=="N":
        typelenstr=type_length[1:] # 3.2
        if "." in type_length:
            deci=typelenstr.index(".")
            x0=typelenstr[:deci]
            x1=typelenstr[deci+1:]
            return float,sum(map(int,(x0,x1)))+1
        else:
            return float,int(type_length[1:])
    elif type=="Y":
        return NotImplementedError("")

def _edit_check(data,ect):
    data=data.to_frame()
    data["_TYPE"]=[isinstance(q,ect[0]) for q in data.iloc[:,0]]
    if all(data._TYPE):
        data["_LEN"]=data.iloc[:,0].str.len()==ect[1]
        if all(data._LEN):
            return True
    return data[~(data._TYPE+data._LEN)]

In [68]:
print(_get_ect(spec.type))

(str, 8)

In [74]:
_edit_check(mh.SUBJID,_get_ect(spec.type))

True

- mh.SUBJID는 문제가 없음

### 후기
- Specsheet에서 KEY가 1이 아닐 때 빈 값이 허용된다.
    - 특히 longitudinal이거나 visit site, visit time에 따라 하나의 테이블에도 long / wide로 다루어야 하는 부분이 혼재한다.
    - 때문에 DOMAIN 등에 따라 거시적으로 categorise 하는 것은 의미가 없다.
        - 최종 데이터 row별로 대응 컬럼과 값을 따르는 조건문 구성이 필요하다.
- 가장 좋은 방법은 분석 또는 당국 규제에 대응되지 않는 CRF 항목이나 데이터는 줄이고, 입력하게 되는 데이터는 CRF 디자인이나 DVP에 따라 최대한 채워넣도록 하는 것이다.